In [ ]:
import cv2
import numpy as np
from OpenCV_Yolo import *
from glob import glob as gl
from matplotlib import pyplot as plt

In [ ]:
# Plate recognition
from modules.clova_text_recognition import predict as plate_recognize


# Plates detection
yolo_v4, classes = load_yolo("yolov4_tiny/cfg/yolov4-tiny-obj.cfg", 
                    "yolov4_tiny/backup/yolov4-tiny-obj_best.weights", 
                    "yolov4_tiny/data/obj.names")

def plate_detect(img, yolo_v4, classes):
    ln = yolo_v4.getLayerNames()
    ln = [ln[i[0] - 1] for i in yolo_v4.getUnconnectedOutLayers()]
    ab_boxes, confidences, classIDs, idxs = make_prediction(
        yolo_v4, ln, classes, img, 0.25 , 0.35)
    output = convert_ouput(ab_boxes, classes, confidences, classIDs, idxs, colors, [])
    return output

In [ ]:
colors = {"plate": (0, 255, 0)}
imgs = gl("Test/*.jpg")
for file in imgs:
    img =cv2.imread(file)
    
    output = plate_detect(img, yolo_v4, classes)
    texts = []
    for out in output:
        x1, y1, x2, y2 = out["pos"]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        crop = img[y1:y2, x1:x2]
        if crop.shape[0]*crop.shape[1]>0:
            text = plate_recognize([crop])[0]
            cv2.putText(img, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 2555, 0), 2)
            texts.append(text)
    plt.figure(figsize=(15, 10))
    plt.imshow(img[:, :, ::-1])
    plt.show()
    print(file)
    print(texts)
    print("#"*30)
    

In [ ]:
%config IPCompleter.greedy=False